In [1]:
import numpy as np
import torch
from itertools import product
from collections import defaultdict
from agents import *
from config import Config

In [2]:
from tqdm import tqdm

def get_next_v(v_bar = 1, sigma_v = 1):
    return v_bar + np.random.normal(scale = sigma_v)

def simulate(
        T = 1000000, config = None,     # simulation time and config file
        continue_simulation = False,    # continue simulation if True
        save_path = None                # path to save the simulation
        ):
    assert config is not None, "Config file required"
    I = config.I
    Np = config.Np
    Nv = config.Nv
    Nx = config.Nx
    sigma_u = config.sigma_u
    if type(continue_simulation) == str:
        log = np.load(continue_simulation, allow_pickle=True).item()
        
        v_hist = np.zeros(T)
        p_hist = np.zeros(T)
        z_hist = np.zeros(T)
        x_hist = np.zeros((I, T))
        # y_hist = np.zeros((I, T))
        u_hist = np.zeros(T)
        profit_hist = np.zeros((I, T))
        t0 = 0

        informed_agents = log["agents"]["informed"]
        noise_agent = log["agents"]["noise"]
        preferred_habitat_agent = log["agents"]["preferred_habitat"]
        market_maker = log["agents"]["market_maker"]

        _state = log["last_state"]

        convergence_counter = log['convergence_counter']
    # elif type(continue_simulation) == dict:
    #     log = continue_simulation
    #     v_hist = log["v"]
    #     p_hist = log["p"]
    #     z_hist = log["z"]
    #     x_hist = log["x"]
    #     y_hist = log["y"]
    #     profit_hist = log["profit"]
    #     t0 = len(v_hist)
    #     informed_agents = log["agents"]["informed"]
    #     noise_agent = log["agents"]["noise"]
    #     preferred_habitat_agent = log["agents"]["preferred_habitat"]
    #     market_maker = log["agents"]["market_maker"]
    #     _state = log["last_state"]

    #     profit_hist = np.concatenate((profit_hist, np.zeros((I, T))), axis=1)
    #     v_hist = np.concatenate((v_hist, np.zeros(T)))
    #     p_hist = np.concatenate((p_hist, np.zeros(T)))
    #     z_hist = np.concatenate((z_hist, np.zeros(T)))
    #     x_hist = np.concatenate((x_hist, np.zeros((I, T))), axis=1)
    #     y_hist = np.concatenate((y_hist, np.zeros((I, T))), axis=1)

    #     convergence_counter = log['convergence_counter']
        
    elif continue_simulation == False:

        market_maker = AdaptiveMarketMaker(config)
        noise_agent = NoiseAgent(config)
        preferred_habitat_agent = PreferredHabitatAgent(config)
        informed_agents = [InformedAgent(config) for _ in range(I)]
        _state = (np.random.choice(Np), np.random.choice(Nv))

        # log histories
        v_hist = np.zeros(T)
        p_hist = np.zeros(T)
        z_hist = np.zeros(T)
        x_hist = np.zeros((I, T))
        # y_hist = np.zeros((I, T))
        u_hist = np.zeros(T)
        profit_hist = np.zeros((I, T))
        t0 = 0
        convergence_counter = 0
    else:
        raise ValueError("Invalid value for continue_simulation")
    for agent in informed_agents:
        agent.convergence_counter = convergence_counter
    if save_path is None:
        save_path = '/Users/moyuanchen/Documents/thesis/data.npy'

    for t in tqdm(range(T), desc="Simulation Progress"):
        yt = []
        _p, _v = informed_agents[0].p_discrete[_state[0]], informed_agents[0].v_discrete[_state[1]]
        v_hist[t+t0] = _v
        p_hist[t+t0] = _p
        _x = []
        for idx, agent in enumerate(informed_agents):
            x = agent.get_action(_state)
            xd = agent.x_discrete[x]
            yt.append(xd)
            _x.append(x)

            x_hist[idx, t + t0] = xd
            # y_hist[idx, t + t0] = yt[-1]
        ut = noise_agent.get_action()
        u_hist[t+t0] = ut
        yt_sum = np.sum(yt) + ut
        # print(yt_sum)
        zt = preferred_habitat_agent.get_action(_p)

        z_hist[t+t0] = zt

        
        pt = market_maker.determine_price(yt_sum)
        market_maker.update(_v, _p, zt, yt_sum)
        vt = get_next_v()
        next_state = informed_agents[0].continuous_to_discrete(pt, vt)
        for idx, agent in enumerate(informed_agents):
            reward = (_v - pt) * yt[idx]
            agent.update(_state, _x[idx], reward, next_state)
            profit_hist[idx, t + t0] = reward

        _state = next_state
    convergence = min([agent.convergence_counter for agent in informed_agents])
    log = {
        "v": v_hist,
        "p": p_hist,
        "z": z_hist,
        "x": x_hist,
        "u": u_hist,
        "profit": profit_hist,
        "last_state": _state,
        "convergence_counter": convergence
    }
    agents = {
        "informed": informed_agents,
        "noise": noise_agent,
        "preferred_habitat": preferred_habitat_agent,
        "market_maker": market_maker
    }
    log["agents"] = agents
    np.save(save_path, log)
    # print(max_c)
    return log, agents

#### Simulate $\sigma_u = 0.1$

In [4]:
config = Config()
config.sigma_u = 0.1

_save_path = '/Users/moyuanchen/Documents/thesis/sim_3/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_3/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_3/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [01:59<00:00, 4189.86it/s]


Partition 1 completed, convergence counter: 1


Simulation Progress: 100%|██████████| 500000/500000 [02:07<00:00, 3926.28it/s]


Partition 2 completed, convergence counter: 33


Simulation Progress: 100%|██████████| 500000/500000 [02:12<00:00, 3780.02it/s]


...

Partition 31 completed, convergence counter: 657908


Simulation Progress: 100%|██████████| 500000/500000 [02:36<00:00, 3203.71it/s]


Partition 32 completed, convergence counter: 909379


Simulation Progress: 100%|██████████| 500000/500000 [02:25<00:00, 3444.92it/s]


Partition 33 completed, convergence counter: 1169752


In [5]:
config = Config()
config.sigma_u = 100

_save_path = '/Users/moyuanchen/Documents/thesis/sim_4/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_4/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_4/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:22<00:00, 3500.73it/s]


Partition 1 completed, convergence counter: 17


Simulation Progress: 100%|██████████| 500000/500000 [02:34<00:00, 3241.04it/s]


Partition 2 completed, convergence counter: 7


Simulation Progress: 100%|██████████| 500000/500000 [02:36<00:00, 3195.91it/s]


...

Partition 75 completed, convergence counter: 532550


Simulation Progress: 100%|██████████| 500000/500000 [02:29<00:00, 3337.76it/s]


Partition 76 completed, convergence counter: 912505


Simulation Progress: 100%|██████████| 500000/500000 [02:20<00:00, 3566.09it/s]

Partition 77 completed, convergence counter: 1294355


### $\sigma = 0.01, 1, 10$

In [3]:
config = Config()
config.sigma_u = 0.01

_save_path = '/Users/moyuanchen/Documents/thesis/sim_5/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_5/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_5/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [01:52<00:00, 4437.36it/s]


Partition 1 completed, convergence counter: 12


Simulation Progress: 100%|██████████| 500000/500000 [02:01<00:00, 4101.41it/s]


Partition 2 completed, convergence counter: 9


Simulation Progress: 100%|██████████| 500000/500000 [02:12<00:00, 3775.09it/s]


...

Partition 35 completed, convergence counter: 428134


Simulation Progress: 100%|██████████| 500000/500000 [02:50<00:00, 2927.33it/s]


Partition 36 completed, convergence counter: 715078


Simulation Progress: 100%|██████████| 500000/500000 [02:48<00:00, 2975.40it/s]


Partition 37 completed, convergence counter: 1010722


In [4]:
config = Config()
config.sigma_u = 1

_save_path = '/Users/moyuanchen/Documents/thesis/sim_6/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_6/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_6/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:57<00:00, 2811.92it/s]


Partition 1 completed, convergence counter: 3


Simulation Progress: 100%|██████████| 500000/500000 [02:26<00:00, 3422.13it/s]


Partition 2 completed, convergence counter: 19


Simulation Progress: 100%|██████████| 500000/500000 [02:23<00:00, 3487.60it/s]


...

Partition 35 completed, convergence counter: 717031


Simulation Progress: 100%|██████████| 500000/500000 [02:36<00:00, 3200.48it/s]


Partition 36 completed, convergence counter: 996457


Simulation Progress: 100%|██████████| 500000/500000 [02:41<00:00, 3086.98it/s]

Partition 37 completed, convergence counter: 1286052


In [5]:
config = Config()
config.sigma_u = 10

_save_path = '/Users/moyuanchen/Documents/thesis/sim_7/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_7/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_7/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [03:07<00:00, 2663.45it/s]


Partition 1 completed, convergence counter: 1


Simulation Progress: 100%|██████████| 500000/500000 [02:32<00:00, 3273.45it/s]


Partition 2 completed, convergence counter: 21


Simulation Progress: 100%|██████████| 500000/500000 [02:47<00:00, 2989.52it/s]


...

Partition 39 completed, convergence counter: 575357


Simulation Progress: 100%|██████████| 500000/500000 [02:42<00:00, 3075.22it/s]


Partition 40 completed, convergence counter: 864369


Simulation Progress: 100%|██████████| 500000/500000 [02:21<00:00, 3545.05it/s]

Partition 41 completed, convergence counter: 1160388


In [6]:
config = Config()
config.sigma_u = 1000

_save_path = '/Users/moyuanchen/Documents/thesis/sim_8/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_8/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_8/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:07<00:00, 3927.63it/s]


Partition 1 completed, convergence counter: 17


Simulation Progress: 100%|██████████| 500000/500000 [02:10<00:00, 3840.22it/s]


Partition 2 completed, convergence counter: 7


Simulation Progress: 100%|██████████| 500000/500000 [02:11<00:00, 3814.52it/s]


...

Partition 489 completed, convergence counter: 85


Simulation Progress: 100%|██████████| 500000/500000 [10:02<00:00, 830.36it/s] 


Partition 490 completed, convergence counter: 2070


Simulation Progress:  39%|███▉      | 195339/500000 [02:16<03:32, 1436.13it/s]


KeyboardInterrupt: 

In [ ]:
config = Config()
config.sigma_u = 10000

_save_path = '/Users/moyuanchen/Documents/thesis/sim_9/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_9/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_9/config.npy', config)

In [7]:
config = Config()
config.sigma_u = 1e-3

_save_path = '/Users/moyuanchen/Documents/thesis/sim_10/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_10/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_10/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:26<00:00, 3420.33it/s]


Partition 1 completed, convergence counter: 0


Simulation Progress: 100%|██████████| 500000/500000 [02:18<00:00, 3608.36it/s]


Partition 2 completed, convergence counter: 12


Simulation Progress: 100%|██████████| 500000/500000 [02:22<00:00, 3499.97it/s]


...

Partition 33 completed, convergence counter: 532532


Simulation Progress: 100%|██████████| 500000/500000 [02:19<00:00, 3574.43it/s]


Partition 34 completed, convergence counter: 800519


Simulation Progress: 100%|██████████| 500000/500000 [02:39<00:00, 3130.33it/s]

Partition 35 completed, convergence counter: 1076099


In [8]:
config = Config()
config.sigma_u = 1e-4

_save_path = '/Users/moyuanchen/Documents/thesis/sim_11/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_11/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_11/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:23<00:00, 3478.58it/s]


Partition 1 completed, convergence counter: 20


Simulation Progress: 100%|██████████| 500000/500000 [02:39<00:00, 3132.58it/s]


Partition 2 completed, convergence counter: 38


Simulation Progress: 100%|██████████| 500000/500000 [02:47<00:00, 2979.96it/s]


...

Partition 33 completed, convergence counter: 667475


Simulation Progress: 100%|██████████| 500000/500000 [02:34<00:00, 3239.92it/s]


Partition 34 completed, convergence counter: 936450


Simulation Progress: 100%|██████████| 500000/500000 [02:29<00:00, 3347.49it/s]

Partition 35 completed, convergence counter: 1215301


In [10]:
config = Config()
config.sigma_u = 1e-5

_save_path = '/Users/moyuanchen/Documents/thesis/sim_12/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_12/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_12/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:07<00:00, 3927.16it/s]


Partition 1 completed, convergence counter: 22


Simulation Progress: 100%|██████████| 500000/500000 [02:29<00:00, 3336.48it/s]


Partition 2 completed, convergence counter: 2


Simulation Progress: 100%|██████████| 500000/500000 [02:31<00:00, 3296.62it/s]


...

Partition 29 completed, convergence counter: 670784


Simulation Progress: 100%|██████████| 500000/500000 [02:45<00:00, 3017.83it/s]


Partition 30 completed, convergence counter: 905855


Simulation Progress: 100%|██████████| 500000/500000 [02:17<00:00, 3629.76it/s]


Partition 31 completed, convergence counter: 1149560


In [11]:
config = Config()
config.sigma_u = 1e-6

_save_path = '/Users/moyuanchen/Documents/thesis/sim_13/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_13/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_13/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:17<00:00, 3648.20it/s]


Partition 1 completed, convergence counter: 74


Simulation Progress: 100%|██████████| 500000/500000 [02:16<00:00, 3669.69it/s]


Partition 2 completed, convergence counter: 5


Simulation Progress: 100%|██████████| 500000/500000 [02:28<00:00, 3369.98it/s]


...

Partition 35 completed, convergence counter: 458919


Simulation Progress: 100%|██████████| 500000/500000 [02:45<00:00, 3014.74it/s]


Partition 36 completed, convergence counter: 745838


Simulation Progress: 100%|██████████| 500000/500000 [02:19<00:00, 3584.56it/s]

Partition 37 completed, convergence counter: 1043197


In [18]:
sigma_list = [10**(-i) for i in range(7, 13)]
import os
sim_counter = 13
for sigma in sigma_list:
    save_folder = f'/Users/moyuanchen/Documents/thesis/sim_{sim_counter}/'
    sim_counter += 1
    try:
        os.mkdir(save_folder)
    except FileExistsError:
        pass
    config = Config()
    config.sigma_u = sigma

    _save_path = save_folder + f'data_part_0.npy'
    log, agents = simulate(config = config, T = 500000, save_path=_save_path)
    convergence_threshold = 1000000
    convergence = 0
    partitions = 1
    while convergence < convergence_threshold:
        save_path = save_folder + 'data_part_{partitions}.npy'
        log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
        _save_path = save_path
        
        convergence = log['convergence_counter']
        print(f"Partition {partitions} completed, convergence counter: {convergence}")
        partitions += 1

    # save config
    np.save(save_folder + 'config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:04<00:00, 4013.97it/s]


Partition 1 completed, convergence counter: 11


Simulation Progress: 100%|██████████| 500000/500000 [02:19<00:00, 3595.68it/s]


Partition 2 completed, convergence counter: 49


Simulation Progress: 100%|██████████| 500000/500000 [02:10<00:00, 3831.67it/s]


...

Partition 32 completed, convergence counter: 480547


Simulation Progress: 100%|██████████| 500000/500000 [02:14<00:00, 3718.94it/s]


Partition 33 completed, convergence counter: 739721


Simulation Progress: 100%|██████████| 500000/500000 [02:19<00:00, 3591.67it/s]

Partition 34 completed, convergence counter: 1006650


In [19]:
print([i for i in range(-5,5)])

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4]


In [ ]:
sigma_list = [10**(i) for i in range(-10, 4)]
import os
sim_counter = 0
for sigma in sigma_list:
    save_folder = f'/Users/moyuanchen/Documents/thesis/5mconv/sim_{sim_counter}/'
    sim_counter += 1
    try:
        os.mkdir(save_folder)
    except FileExistsError:
        pass
    config = Config()
    config.sigma_u = sigma

    _save_path = save_folder + f'data_part_0.npy'
    log, agents = simulate(config = config, T = 500000, save_path=_save_path)
    convergence_threshold = 5000000
    convergence = 0
    partitions = 1
    while convergence < convergence_threshold:
        save_path = save_folder + f'data_part_{partitions}.npy'
        log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
        _save_path = save_path
        
        convergence = log['convergence_counter']
        print(f"Partition {partitions} completed, convergence counter: {convergence}")
        partitions += 1

    # save config
    np.save(save_folder + 'config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:03<00:00, 4060.08it/s]


Partition 1 completed, convergence counter: 2


Simulation Progress: 100%|██████████| 500000/500000 [02:01<00:00, 4110.34it/s]


Partition 2 completed, convergence counter: 42


Simulation Progress: 100%|██████████| 500000/500000 [01:58<00:00, 4223.98it/s]


...

Simulation Progress: 100%|██████████| 500000/500000 [02:03<00:00, 4051.15it/s]


Partition 20 completed, convergence counter: 152815


Simulation Progress: 100%|██████████| 500000/500000 [02:03<00:00, 4052.85it/s]


Partition 21 completed, convergence counter: 26727


Simulation Progress:  55%|█████▌    | 275021/500000 [01:12<00:59, 3770.44it/s]

### Continue Simulation after Convergence

In [6]:
config = Config()
config.sigma_u = 0.1

_save_path = '/Users/moyuanchen/Documents/thesis/sim_1/converged/data.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path, continue_simulation=)

SyntaxError: expected argument value expression (1705007820.py, line 5)